# Métodos de Runge-Kutta

# Samuel Amat, Uziel Linares

En este notebook, veremos unos métodos numéricos para integrar ODEs llamados **métodos de Runge-Kutta** (por sus inventores). Versiones de estos métodos son unos de los más comúnmente utilizados hoy día para integrar EDOs.

Considera la EDO

$$\dot{x}(t) = f(x(t), t) \qquad (1)$$

**[1]** Dado $x(t_0)$, queremos encontrar $x(t_0 + h)$. 

(i) Desarrolla $x(t_0 +h)$ en una serie de Taylor, incluyendo los términos hasta segundo orden en $h$. Para calcular $\ddot{x}$, deriva la ecuación (1) con respecto a $t$. Nota que $f$ es una **función de dos variables**.

(ii) ¿A qué corresponde el método de Euler?

[1] (i)
La expansión en serie de Taylor de $x(t_0 + h)$ hasta orden dos es 
$$
x(t_0 + h) \approx x(t_0) + \dot{x}(t_0)h + \dfrac{1}{2!}\ddot{x}(t_0)h^2
$$ 
Luego de (1) se tiene
$$
\ddot{x}(t_0) = \dfrac{df}{dx}\dfrac{dx}{dt} + \dfrac{\partial f}{\partial t}\\
 = \dfrac{\partial f}{\partial x}\dot{x}(t_0) + \dfrac{\partial f}{\partial t}
$$
Por lo que 
$$
x(t_0 + h) \approx x(t_0) + \dot{x}(t_0)h + \dfrac{1}{2!}\{\dfrac{\partial f(x_0, t_0)}{\partial x}\dot{x}(t_0) + \dfrac{\partial f(x_0, t_0)}{\partial t}\}h^2
$$

De manera que el método de Euler corresponde a la serie de Taylor en primer orden.

Se pueden derivar métodos, llamados **métodos de Taylor**, que calculan explícitamente las derivadas de $f$. Los métodos de Runge-Kutta utilizan una idea diferente: evaluamos $f$ varias veces, posiblemente en distintos lugares, y tomamos una combinación lineal de estas evaluaciones para reproducir la expansión de Taylor a diferentes órdenes.

Veremos un par de métodos de Runge-Kutta **explícitos**, es decir, en los cuales no es necesario de resolver una ecuación no-lineal (como ocurre en el método de Euler implícito).

**[2]** Para entender la idea de los métodos RK, regresemos al método de Euler para atrás. Si $t_n$ son los nodos en donde aproximamos la solución, y $x_n$ los valores aproximados, entonces tenemos lo siguiente (que obtenemos al aproximar la integral mediante la regla del trapecio):

$$x_{n+1} = x_n + \frac{h}{2} \left[ f(x_n, t_n) + f(x_{n+1}, t_{n+1}) \right] \qquad (2).$$

Para convertir esta ecuación implícita en un método Runge-Kutta, tomemos una *aproximación* de $f(x_{n+1})$, al utilizar un *paso de Euler*. En general, los métodos de Runge-Kutta incorporan varios pasos de Euler, que pueden ser de distintos tamaños.

(i) Escribe la ecuación de un paso de Euler para $x_{n+1}$ en términos de $x_n$.

(ii) Inserte ese paso de Euler en la ecuación (2). Expande en potencias de $h$ hasta segundo orden. Demuestra que recupera la expansión de Taylor de $x(t+h)$ a segundo orden. Este método se llama el **método de Euler modificado**.

[2] i-ii) 

El paso de Euler para $x_{n + 1}$ es:

$x_{n + 1} = x_n + hf(x_n, t_n)$

Insertando ésto en la ecuación $2$ se obtiene:

$$x_{n+1} = x_n + \frac{h}{2} \left[ f(x_n, t_n) + f(x_n + hf(x_n, t_n), t_{n+1}) \right] \qquad (2).$$

Una alternativa es el tomar un paso de Euler en todo el intervalo de tamaño $h$, pero utilizando una mejor aproximación de la derivada en el intervalo. Para hacerlo, se toma un primer paso de Euler hasta la *mitad* del camino entre $t_n$ y $t_{n+1}$, es decir una "distancia" $h/2$ en el tiempo, y se evalúa ahí $f(x(t+h/2), t+h/2)$. Este valor luego se utiliza como la aproximación de $\dot{x}$ sobre el intervalo en otro paso de Euler. Este método se llama el **método del punto medio**.

(iii) Implementa estos dos métodos. (Los dos son métodos de tipo Runge-Kutta). Confirma numéricamente cuáles son sus respectivas tasas de convergencia, y compara visualmente los resultados con los métodos de Euler y Euler para atrás para distintos sistemas.

In [2]:
using Plots
using Interact
gr()

Plots.GRBackend()

In [3]:
 function intervalo(t0, x0, h, tn)
    ts = t0:h:tn
    x = zeros(ts)
    x[1] = x0
    return ts, x
end

intervalo (generic function with 1 method)

In [4]:
function euler_mod(f, h, t0, x0, tn=10)
    ts, x = intervalo(t0, x0, h, tn)
    for i in 1:length(ts) - 1
        x[i+1] = x[i] + ((h/2) * (f(x[i], ts[i]) + f((h * f(x[i], ts[i])) + x[i], ts[i + 1])))
    end
    return ts, x
end

euler_mod (generic function with 2 methods)

In [5]:
function punto_medio(f, h, t0, x0, tn=10)
    ts, x = intervalo(t0, x0, h, tn)
    for i in 1:length(ts)-1
        xm = ((h/2) * f(x[i], ts[i])) + x[i]
        x[i+1] = (h * f(xm, ts[i+1])) + x[i]
    end
    return ts, x 
end

punto_medio (generic function with 2 methods)

Se comparan los métodos para las ecuaciones
$$
\dot{x}(t) = x(t)
$$

Y

$$
\dot{x}(t) = x(t)(1 - 2t)
$$

In [6]:
# [1] (iii)
function euler(f, h, t0, x0, tmax=10)
    ts = t0:h:10
    x = zeros(ts)
    x[1] = x0
    for i in 1:length(ts)-1
        x[i+1] = (h * f(x[i], ts[i])) + x[i]
    end
    return ts, x
end 

euler (generic function with 2 methods)

In [28]:
# [2] iii)
t0 = 0
x0 = 1
h = 0.1
s(x, t) = x * (1 - 2t)
t1, x1 = euler_mod(s, h, t0, x0)
t2, x2 = punto_medio(s, h, t0, x0)

(0.0:0.1:10.0, [1.0, 1.084, 1.15164, 1.19909, 1.22355, 1.22355, 1.19908, 1.1516, 1.08388, 0.999773  …  4.68827e-19, 3.54621e-19, 2.73484e-19, 2.15067e-19, 1.72484e-19, 1.41092e-19, 1.17727e-19, 1.00209e-19, 8.70218e-20, 7.71013e-20])

In [29]:
function convergencia(f, sol, h, t0, x0)
    t1, x1 = euler_mod(f, h, t0, x0)
    t2, x2 = punto_medio(f, h, t0, x0)
    dif1 = norm(sol.(t1) - x1)
    dif2 = norm(sol.(t2) - x2)
    return dif1, dif2
end

convergencia (generic function with 1 method)

In [30]:
scatter(t1, x1, label="Euler Modificado", xlims=(0,2))
scatter!(t2, x2, label="Punto Medio")
plot!(p -> exp(p-p^2))
#scatter!(t3, x3, label="Euler")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 1.2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Euler Modificado 
 
 
 
 
 Punto Medio 
 
 
 
 y3

In [31]:
s(x, t) = x * (1 - 2t)
sol(t) = exp(t - t^2)
t0 = 0
x0 = 1
hs1 = []
hs2 = []
for h in 0.1:.001:1
    d1, d2 = convergencia(s, sol, h, t0, x0)
    push!(hs1, d1)
    push!(hs2, d2)
end

In [32]:
scatter(0.1:.001:1, log.(hs2), xlim=(0,.3))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
 
 0.20 
 
 
 0.25 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 20 
 
 
 25 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

## Runga-Kutta de más alto orden

Se pueden derivar métodos de Runge-Kutta de más alto orden, siempre con la meta de reproducir la expansión de Taylor de $x(t+h)$ a cada vez más alto orden. Sin embargo, los cálculos se vuelven complicados, por lo cual no veremos aquí los detalles. (Se pueden encontrar en el libro de Burden & Faires, por ejemplo).

**[3]** Uno de los más utilizados es "RK4", de cuarto orden. Encuentra las ecuaciones para este método e impleméntalo para el caso de ecuaciones arbitrarias en forma vectorial. (Puedes empezar con el caso escalar.) 

Encuentra numéricamente su tasa de convergencia y compara visualmente el resultado con los demás métodos.

In [12]:
# Runge Kutta 4th escalar
function runge_kutta4(f, h, t0, x0, tn=10)
    ts, x = intervalo(t0, x0, h, tn)
    for i in 1:length(ts) - 1
        k1 = f(x[i], ts[i])
        k2 = f(x[i] + (h/2)*k1, ts[i] + h/2)
        k3 = f(x[i] + (h/2)*k2, ts[i] + h/2)
        k4 = f(x[i] + h*k3, ts[i] + h)
        x[i+1] = x[i] + (h/6)*(k1 + 2k2 + 2k3 + k4)
    end
    return ts, x
end

runge_kutta4 (generic function with 2 methods)

In [13]:
t0 = 0
x0 = 1
α = 1
h = 0.1
t, x = runge_kutta4((x,t) -> -α*x, h, t0, x0)
scatter(t, x, label="Numérico", xlabel="Tiempo")
plot!(x -> exp(-α*x), label="Analítico")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 1.2 
 
 
 Tiempo 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Numérico 
 
 
 
 Analítico

In [14]:
# Runge Kutta vectorial

function paso_runge_kutta(f, h, t0, x0)
    k1 = f(x0, t0)
    k2 = f(x0 + (h/2)*k1, t0 + h/2)
    k3 = f(x0 + (h/2)*k2, t0 + h/2)
    k4 = f(x0 + h*k3, t0 + h)
    return (x0 + (h/6)*(k1 + 2k2 + 2k3 + k4))
end


function runge_4_vectorial(f, t0, x0, h=0.01; tn=10)
    ts = 0:h:tn
    m_dim = length(x0)
    n_dim = length(ts)
    xs = zeros((m_dim, n_dim))
    xs[:, 1] = x0
    for i in 1:length(ts)-1
        xs[:, i+1] = paso_runge_kutta(f, h, ts[i], xs[:, i])
    end
    return ts, xs
end

runge_4_vectorial (generic function with 2 methods)

In [15]:
# [5] (iV)
# Caida libre
grav = 9.8
sis1(x, t) = [x[2]; -grav]
x0 = [100; 0]
t0 = 0
h = 0.1
t, sols = runge_4_vectorial(sis1, t0, x0, h, tn=10)

(0.0:0.1:10.0, [100.0 99.951 … -380.249 -390.0; 0.0 -0.98 … -97.02 -98.0])

In [16]:
caida_libre(t, v0, y0) = -(9.8/2)t^2 + v0*t + y0

caida_libre (generic function with 1 method)

In [17]:
exacta = caida_libre.(t, 0, 100)
scatter(t, exacta, label="Analítica", xlabel="Tiempo", ylabel="Altura")
plot!(t, sols[1,:], label="Numérica")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 -400 
 
 
 -300 
 
 
 -200 
 
 
 -100 
 
 
 0 
 
 
 100 
 
 
 Tiempo 
 
 
 Altura 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Analítica 
 
 
 
 Numérica

## Paso adaptativo

Hasta ahora, en todas las integraciones de EDOs que hemos hecho, ha habido un tamaño de paso fijo, que es un parámetro que pasamos a la función `RK4` etc.

Pero surge una pregunta: ¿cómo se debe escoger el tamaño del paso? (Seguramente ¡ya se te ha ocurrido esta pregunta!) La respuesta dependerá de la función $\mathbf{f}$ que estemos integrando: si $\mathbf{f}$ cambia rápido, debemos usar un paso más chiquito para resolver los detalles de la función; si $\mathbf{f}$ cambia más lentamente, podemos utilizar un paso más grande. 

El problema es que ¡sólo podemos saber qué tan rápido varía la función cuando estemos en medio de la propia integración numérica!
La solución es utilizar un método *adaptativo*: el método mismo tiene (cierto) control del tamaño de paso, el cual *se irá cambiando de manera automática* para tomar en cuenta la propia tasa de cambio de la función.

Por esta razón, (casi) *nunca* se deberían utilizar los métodos simples y no-adaptativos como Euler y RK4 en la práctica.

## Euler adaptativo 

Dado que este tema se puede volver complicado, consideremos el método de Euler por simplicidad. Queremos resolver 

$$\dot{x} = f(x),$$

y tenemos

$$x_{n+1} = x_n + h \, f(x_n) + \epsilon_1(h), (*)$$

donde $\epsilon_1(h) = C \, h^2$ es el error de un paso.
Aquí, hemos supuesto que $C$ no depende de $h$. Esto no es realmente cierto (¿por qué?), pero facilita el cálculo.

Para ciertos tipos de función $f$ (¿cuáles? -- ¿qué otra forma podríamos utilizar para el término del error?), el término del error será grande.
¿Cómo podemos *estimar* el tamaño de este término? Una idea es el de tomar *dos* pasos, de tamaño $h/2$.

**[4]** (i) Encuentra la expresión para $x_n$ si se toman dos pasos de tamaño $h/2$, 
donde $x_{n+\frac{1}{2}}$ es el lugar intermedio.
Substrae los dos resultados del método de Euler para encontrar el tamaño del error $\epsilon$.

Si $\epsilon < \mathrm{tol}$, una cierta tolerancia que imponemos, entonces el paso es exitoso, y actualizamos las variables. En este caso, la función está variando lentamente, así que podemos *incrementar* el tamaño del paso. 
Si no es exitoso, reducimos la tolerancia. En los dos casos, podemos actualizar según una regla de la forma

$$ h' = 0.9 h \, \frac{\mathrm{tol}}{|\epsilon|}.$$


(ii) Implementa un método adaptativo de Euler.

(iii) Prúebalo para un sistema que hemos estudiado en el cual fracasa Euler. ¿Ayuda?

4)i)

Tenemos una primera aproximación:

$$x_{n+1}^{(1)} = x_n + h \, f(x_n, t_n) + \epsilon_1(h)$$

donde, se define el error como: 

$$\epsilon_1 =  x(t_{n+1}) - x_{n+1}^{(1)}\: \implies \: x_{n+1}^{(1)} = x(t_{n+1}) - \epsilon_1 \qquad (1).$$

con $x(t_{n+1})$ la solución exacta al tiempo $t_{n+1}$

y donde, además $\epsilon_1(h) = C \, h^2$

Ahora usamos dos pasos de Euler de tamaño 1/2, entonces:

$x_{n+1/2} = x_n + \dfrac{h}{2} f(x_n, t_n) $

$x_{n+1}^{(2)} = x_{n+1/2} + \dfrac{h}{2} f(x_{n+1/2}, t_{n+1/2} )$

$$\epsilon_2 = C\left( \dfrac{h}{2} \right) ^2 + C\left( \dfrac{h}{2} \right) ^2 = 2C\left( \dfrac{h}{2} \right) ^2 = \dfrac{1}{2} Ch^2 = \dfrac{1}{2}\epsilon_1 \qquad (2).$$

$$x_{n+1}^{(2)} = x(t_{n+1}) - \epsilon_2  \qquad (3).$$

Tomando la resta $x_{n+1}^{(2)} - x_{n+1}^{(1)} $ usando $1$ y $2$ y $3$ se obtiene

$$\boxed{\epsilon_1 = 2 \left( x_{n+1}^{(2)} - x_{n+1}^{(1)} \right) \qquad(4).}$$

In [18]:
#4[ii]
function euler_adaptativo(f, h, t0, x0, tol, tmax=10)
    ts = t0:h:10
    x1 = zeros(ts)
    x2 = zeros(ts)
    x1[1] = x0
    x2[2] = x0
    ϵ = 0
    for i in 1:length(ts)-1
        if ϵ >= tol
            h = 0.9*h*(tol/abs(ϵ))
        end
        x1[i+1] = (h * f(x1[i], ts[i])) + x[i]
        x_intermedio = ((h/2) * f(x2[i], ts[i])) + x2[i]
        x2[i+i] = (h * f(x_intermedio, ts[i+1])) + x2[i]
        ϵ = 2*(x2[i+1] - x1[i+1])
    end
    return ts, x1
end 

euler_adaptativo (generic function with 2 methods)

## Métodos de Runge-Kutta adaptativos 

Supongamos que hiciéramos la misma idea para Runge-Kutta 4.

**[5]** ¿Cuántas evaluaciones de la función $f$ se requieren para llevar a cabo un paso de tamaño $h$?

Dado que esto puede ser caro, hay una mejor solución. Resulta que hay métodos de Runge-Kutta ("embedded") tales que podemos utilizar las *mismas* evaluaciones de la función $f$ (en los mismos lugares del intervalo $[t, t+h]$), y nos da *dos* estimados diferentes de $x(t+h)$, con dos órdenes de error distintos. Esto se puede aplicar de la misma forma para controlar el tamaño de paso, pero con menos evaluaciones de $f$ en cada paso.

Este método, y otros parecidos, es uno de los que se suele utilizar para cálculos serios.

**[6]** ** (Opcional) Implementa el método "RK45" (Runge-Kutta-Fehlberg), que mezcla un método de 4o. y de 5o. orden.